In [ ]:
# default_exp lightningreapp

In [ ]:
# export
import pickle
import pandas as pd
import pytorch_lightning as lit
import torch
from pytorch_lightning.metrics.functional import r2score, explained_variance
from torch import nn, optim
from torch.nn import functional as F
from transformers import AutoModel

default_model_name = "distilbert-base-uncased-finetuned-sst-2-english"


class LightningReapp(lit.LightningModule):
    def __init__(self, config, pretrained_model_name=default_model_name):
        super().__init__()

        # Set model hyperparams
        self.lr = config.get("lr", 1e-3)
        self.num_embedding_layers = config.get('num_embedding_layers', 1)
        self.save_hyperparameters()

        # Initialize a pretrained model
        self.bert = AutoModel.from_pretrained(
            pretrained_model_name,
            output_hidden_states = True
        )

        self.feature_dim = self.bert.config.dim

        # Turn off autograd for bert encoder
        for param in self.bert[:-self.num_embedding_layers].parameters():
            param.requires_grad = False

        self.distance_scorer = nn.Sequential(
            nn.Linear(self.feature_dim * self.num_embedding_layers, self.feature_dim),
            nn.Linear(self.feature_dim, 1),
            nn.ReLU(),
        )

        self.pooler = nn.AdaptiveAvgPool1d(1)



        # define metrics
        self.train_loss = lit.metrics.MeanSquaredError()
        self.val_loss = lit.metrics.MeanSquaredError()

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids, attention_mask, output_hidden_states=True)
        last_hidden_states = output.hidden_states[-self.num_embedded_layers:]
        states_combined = torch.cat(last_hidden_states, dim=2)
        avg = self.pooler(states_combined.permute(0,2,1).squeeze())
        # avg = get_avg_masked_encoding(last_hidden_state, attention_mask)
        out = self.distance_scorer(avg).squeeze()
        return out

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

    def training_step(self, batch, batch_idx):
        # destructure batch
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        score = batch["score"]
        # Compute the loss
        output = self(input_ids, attention_mask)
        loss = self.train_loss(output.sum(dim=1), score)
        self.log("train_loss", loss)
        return {"loss": loss}

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x["loss"] for x in outputs]).mean()
        self.log("train_loss", avg_loss)

    # VALIDATION LOOP
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        expected = batch["score"]
        output = self(input_ids, attention_mask)
        observed = output.sum(dim=1)
        loss = self.val_loss(observed, expected)
        return {
            "val_loss": loss,
            'r2score': r2score(observed, expected),
            'explained_var': explained_variance(observed, expected)
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        r2score = torch.stack([x["r2score"] for x in outputs]).mean()
        explained_var = torch.stack([x["explained_var"] for x in outputs]).mean()

        # calculate spearman's r and pearson's r
        self.log("val_loss", avg_loss)
        self.log('r2score', r2score.item())
        self.log('explained_var', explained_var.item())


    # TESTING LOOP
    def test_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        output = self(input_ids, attention_mask)
        # Eval step
        return {"predict": (batch_idx, output.sum(dim=1))}

    def test_epoch_end(self, outputs):
        dfs = []
        for output in outputs:
            batch_idx, result = output['predict']
            dfs.append((batch_idx, result.cpu().tolist()))
        with open(f"./output_reapp.pkl", 'wb+') as f:
            pickle.dump(dfs, f)


def get_avg_masked_encoding(state: torch.Tensor, attention_mask: torch.Tensor):
    """[summary]
    For B = batch size, L = encoding length, F = feature vector:
    Args:
        state (torch.Tensor): (B, L, F)
        attention_mask (torch.Tensor): (B, L)
    Returns:
        torch.Tensor: (B, F), where L has been masked by `attention_mask` and then averaged.
            Each vector in the batch represents the average feature vector for the masked_encoding
    """
    encodings = state.unbind(dim=0)  # split the batch up
    new_tensors = []
    # TODO: VECTORIZE
    for i in range(len(encodings)):
        # for each element in the encoding dimension, get the elements that aren't padding using the attention_mask
        encoding_mask = attention_mask[i]
        # Find the indices where attention_mask > 0 (where the actual tokens are without padding)
        indices = encoding_mask.nonzero(as_tuple=True)[0]
        masked_encoding = (
            encodings[i].index_select(0, indices).squeeze()
        )  # torch.Size([len(indices), F])
        avg_feature = torch.mean(masked_encoding, dim=0)
        new_tensors.append(avg_feature)

    return torch.stack(new_tensors)

In [ ]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('distilbert-base-uncased', output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

model.config



DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.2.2",
  "vocab_size": 30522
}

In [ ]:
import torch
output = model.forward(model.dummy_inputs['input_ids'])

hidden_states = output.hidden_states
num_hidden_states = 2
last_two = hidden_states[-num_hidden_states:]
## Concatenate the last two states
last_two_concat = torch.cat(last_two, dim=2)
last_two_concat.shape

import torch.nn.functional as F

avg = F.avg_pool1d(
    last_two_concat.permute(0,2,1), 
    kernel_size=5, 
    count_include_pad=False).squeeze()

print(avg.shape)
F.linear(avg, weight=torch.rand(avg.shape[1]))



torch.Size([3, 1536])


tensor([-28.8698, -28.1979, -29.1671], grad_fn=<MvBackward>)

In [ ]:
from nbdev.export import notebook2script
notebook2script('LightningReapp.ipynb')

Converted LightningReapp.ipynb.
